In [2]:
!pip install -U openai-whisper
!pip install -U openai-whisper
!apt-get install -y ffmpeg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 16.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=8e772bb985b73deecc3b12c01e3055cd98616588de6e898f2dd2526a259a364b
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [3]:
!pip install jiwer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 24.7 MB/s eta 0:00:00


In [4]:
import pandas as pd
import requests
from datasets import Dataset, Audio
import os, requests, pandas as pd
from urllib.parse import urlparse, unquote
from datasets import Dataset, Audio,load_dataset
from tqdm import tqdm
import whisper
import librosa
from sklearn.model_selection import train_test_split

import json
import torch
from tqdm import tqdm
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments, WhisperProcessor
from torch.nn.utils.rnn import pad_sequence
from transformers import get_scheduler
from torch.utils.data import Dataset, DataLoader
from transformers import WhisperForConditionalGeneration, WhisperTokenizer, WhisperFeatureExtractor, Trainer, TrainingArguments


In [ ]:
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="hi", task="transcribe")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="hi", task="transcribe")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

In [6]:
drive_link="https://drive.google.com/file/d/141PTltzSwvD26E77q2AJK2VYsRnl9qMS/view?usp=sharing"

In [7]:
file_id = "141PTltzSwvD26E77q2AJK2VYsRnl9qMS"

In [8]:
download_url = f"https://drive.google.com/uc?id={file_id}"

In [16]:
response = requests.get(download_url)
response.raise_for_status()

In [10]:
import io
os.makedirs("/content/drive/MyDrive/joshtalk2", exist_ok=True)

In [11]:
!pip install gdown


In [ ]:
import gdown
import pandas as pd

save_path = "/content/drive/MyDrive/joshtalk2/dataset.pkl"

file_id = "141PTltzSwvD26E77q2AJK2VYsRnl9qMS"
gdown.download(f"https://drive.google.com/uc?id={file_id}", save_path, quiet=False)


dataset_df = pd.read_pickle(save_path)
print(dataset_df.head())


Downloading...
From (original): https://drive.google.com/uc?id=141PTltzSwvD26E77q2AJK2VYsRnl9qMS
From (redirected): https://drive.google.com/uc?id=141PTltzSwvD26E77q2AJK2VYsRnl9qMS&confirm=t&uuid=abb0058b-bd29-42c9-a4c0-9ca69de9c570
To: /content/drive/MyDrive/joshtalk2/dataset.pkl
100%|██████████| 4.69G/4.69G [00:56<00:00, 83.5MB/s]


                                               audio  \
0  [5.660588e-05, 0.000104444494, 6.564668e-05, 3...   
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
2  [5.3611493e-05, 4.5976667e-05, 3.947672e-05, 6...   
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
4  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                                text  \
0  अब काफी अच्छा होता है क्योंकि उनकी जनसंख्या बह...   
1  जी जी जी जी जी । जी जी जी हां उधर हां जी हा हा...   
2  लेकिन हम लोग इसे छुपछुप के लोगों के घर जाकर खे...   
3  जी जी जी जी जी मेरे तो जैसे बहुत सारी यादे हैं...   
4  हां जी पहले बात करते हैं विवाह की तो इस मुवी म...   

                                                info  
0  {'user_id': 245746, 'recording_id': 825780, 'l...  
1  {'user_id': 291038, 'recording_id': 825727, 'l...  
2  {'user_id': 246004, 'recording_id': 988596, 'l...  
3  {'user_id': 93626, 'recording_id': 990175, 'la...  
4  {'user_id': 286851, 'recording_id': 526266, 'l..

In [13]:
from transformers import WhisperTokenizer

tokenizer=WhisperTokenizer.from_pretrained('openai/whisper-small',language='hindi',task='transcribe')

In [14]:
target=dataset_df['text']
type(target[0])



str

In [ ]:
def preprocess_sample(sample):
    
    audio_tensor = torch.tensor(sample['audio'], dtype=torch.float32)
    audio_tensor = audio_tensor / audio_tensor.abs().max() 

    tokenized = tokenizer(sample['text'], padding='max_length', truncation=True, max_length=448, return_tensors='pt')
    labels = tokenized['input_ids'].squeeze()

    return {'input_values': audio_tensor, 'labels': labels}

In [17]:
preprocessed_dataset = [preprocess_sample(dataset_df.iloc[i]) for i in range(len(dataset_df))]


In [18]:
class WhisperHindiDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]


In [19]:

dataset = WhisperHindiDataset(preprocessed_dataset)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [ ]:
class WhisperHindiDataset(Dataset):
    def __init__(self, df, processor, max_length=448):
        self.df = df.reset_index(drop=True)
        self.processor = processor
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        audio = self.df.iloc[idx]['audio'] 
        text = self.df.iloc[idx]['text']    

        
        input_features = self.processor.feature_extractor(
            audio,
            sampling_rate=16000,
            return_tensors="pt"
        ).input_features.squeeze(0)  # (80, T)

        
        labels = self.processor.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        ).input_ids.squeeze(0)

        return {
            "input_features": input_features,
            "labels": labels
        }


In [ ]:
def collate_fn(batch):
    input_features = [item["input_features"].T for item in batch] 
    input_features = pad_sequence(input_features, batch_first=True)
    input_features = input_features.permute(0, 2, 1) 

    labels = [item["labels"] for item in batch]
    labels = pad_sequence(labels, batch_first=True, padding_value=-100)

    return {
        "input_features": input_features,
        "labels": labels,
    }

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (f

In [23]:
train_df, temp_df = train_test_split(dataset_df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)


In [24]:
print(f"Train size: {len(train_df)}")
print(f"Validation size: {len(val_df)}")
print(f"Test size: {len(test_df)}")

Train size: 83
Validation size: 10
Test size: 11


In [25]:
train_dataset = WhisperHindiDataset(train_df, processor)
val_dataset = WhisperHindiDataset(val_df, processor)
test_dataset = WhisperHindiDataset(test_df, processor)


In [26]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)


In [27]:
train_dataset = WhisperHindiDataset(dataset_df, processor)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)


In [ ]:
batch = next(iter(train_loader))
print("input_features:", batch["input_features"].shape)  
print("labels:", batch["labels"].shape)

input_features: torch.Size([4, 80, 3000])
labels: torch.Size([4, 448])


In [29]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)


In [30]:
from torch.optim import AdamW


In [31]:
optimizer = AdamW(model.parameters(), lr=1e-5)
epochs = 3
num_training_steps = len(train_loader) * epochs
lr_scheduler = get_scheduler("linear", optimizer, num_warmup_steps=500, num_training_steps=num_training_steps)


In [ ]:
for epoch in range(epochs):

    model.train()
    total_loss = 0
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        input_features = batch["input_features"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_features=input_features, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        total_loss += loss.item()
        loop.set_description(f"Epoch {epoch+1} [Training]")
        loop.set_postfix(loss=loss.item())

    avg_train_loss = total_loss / len(train_loader)
    print(f"\nEpoch {epoch+1} - Avg Training Loss: {avg_train_loss:.4f}")

    
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_features = batch["input_features"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_features=input_features, labels=labels)
            val_loss += outputs.loss.item()
    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch {epoch+1} - Avg Validation Loss: {avg_val_loss:.4f}")



Epoch 1 [Training]: 100%|██████████| 26/26 [01:03<00:00,  2.45s/it, loss=2.19]



Epoch 1 - Avg Training Loss: 1.8501
Epoch 1 - Avg Validation Loss: 1.8200


Epoch 2 [Training]: 100%|██████████| 26/26 [01:06<00:00,  2.55s/it, loss=1.76]



Epoch 2 - Avg Training Loss: 1.6919
Epoch 2 - Avg Validation Loss: 1.6862


Epoch 3 [Training]: 100%|██████████| 26/26 [01:07<00:00,  2.58s/it, loss=1.36]



Epoch 3 - Avg Training Loss: 1.5866
Epoch 3 - Avg Validation Loss: 1.5914


In [33]:
model.save_pretrained("/content/drive/MyDrive/sample_data/joshtalk2")
processor.save_pretrained("/content/drive/MyDrive/sample_data/joshtalk2")
print("Fine-tuned model saved!")

/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:4034: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Fine-tuned model saved!


In [34]:
model = WhisperForConditionalGeneration.from_pretrained("/content/drive/MyDrive/sample_data/joshtalk2")
processor = WhisperProcessor.from_pretrained("/content/drive/MyDrive/sample_data/joshtalk2")



In [ ]:

model.to(device)
model = model.to(device) 
print(next(model.parameters()).device)  


cuda:0


In [ ]:
model.eval()
predictions = []
references = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating Fine-Tuned Model"):
      
        input_features = batch["input_features"].to(device)
        labels = batch["labels"].to(device)

        generated_tokens = model.generate(input_features.to(device))
        predicted_texts = processor.tokenizer.batch_decode(
            generated_tokens.cpu(), skip_special_tokens=True
        )

        
        label_texts = processor.tokenizer.batch_decode(
            labels.cpu(), skip_special_tokens=True
        )

        predictions.extend(predicted_texts)
        references.extend(label_texts)


Evaluating Fine-Tuned Model:   0%|          | 0/3 [00:00<?, ?it/s]Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.
`generation_config` default values have been modified to match model-specific defaults: {'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15

In [37]:
from jiwer import wer

In [38]:
final_wer = wer(references, predictions)
print(f"\nFinal Word Error Rate (WER) - Fine-Tuned Model: {final_wer:.4f}")



Final Word Error Rate (WER) - Fine-Tuned Model: 0.9955


In [39]:
model_name = "openai/whisper-small"

In [40]:
baseline_model = WhisperForConditionalGeneration.from_pretrained(model_name)
baseline_model.to(device)
baseline_model.eval()

baseline_predictions = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating Baseline Model"):
        input_features = batch["input_features"].to(device)
        outputs = baseline_model.generate(input_features)
        predicted_texts = processor.tokenizer.batch_decode(outputs, skip_special_tokens=True)
        baseline_predictions.extend(predicted_texts)

baseline_wer = wer(references, baseline_predictions)
print(f"Baseline Whisper-small WER: {baseline_wer:.4f}")


Evaluating Baseline Model: 100%|██████████| 3/3 [00:22<00:00,  7.66s/it]

Baseline Whisper-small WER: 1.0469


In [41]:
import pandas as pd

results = pd.DataFrame({
    "Model": ["Pretrained Whisper-small", "Fine-tuned Whisper-small"],
    "WER (Hindi Test Set)": [baseline_wer, final_wer]
})

print("\nWER Comparison:")
print(results.to_string(index=False))


WER Comparison:
                   Model  WER (Hindi Test Set)
Pretrained Whisper-small              1.046931
Fine-tuned Whisper-small              0.995487
